In [1]:
import pandas as pd
import numpy as np
import itertools
from matplotlib import pyplot
# forecast monthly births with random forest
from numpy import asarray
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor

In [2]:
series = read_csv('datacount7.csv', header=0, index_col=0, parse_dates=True, squeeze=True)
train_end_date = '2023-07-01'
train_end_date = pd.to_datetime(train_end_date)
train = series.loc[series.index < train_end_date]
test = series.loc[series.index >= train_end_date]
trainvalues = train.values
trainvalues = trainvalues.reshape(-1, 1)
testvalues = test.values
testvalues = testvalues.reshape(-1, 1)

C:\Users\1\AppData\Local\Temp\ipykernel_21008\818203943.py:1: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  series = read_csv('datacount7.csv', header=0, index_col=0, parse_dates=True, squeeze=True)


In [3]:
def series_to_supervised(data,n_in = 1,n_out = 1,dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols = list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
         cols.append(df.shift(i))
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
    # put it all together
    agg = concat(cols, axis=1)
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
        return agg.values

In [4]:
traindata = series_to_supervised(trainvalues, n_in=5)
testdata = series_to_supervised(testvalues, n_in=5)

In [5]:
def random_forest_forecast(train, testX):
	# transform list into array
	train = asarray(train)
	# split into input and output columns
	trainX, trainy = train[:, :-1], train[:, -1]
	# fit model
	model = RandomForestRegressor(n_estimators=n_estimators,max_depth = max_depth,min_samples_leaf = min_samples_leaf)
	model.fit(trainX, trainy)
	# make a one-step prediction
	yhat = model.predict([testX])
	return yhat[0]

In [6]:
def walk_forward_validation(traindata,testdata, n_test,n_estimators,max_depth,min_samples_leaf):
    predictions = list()
    history = [x for x in traindata]
    for i in range(len(testdata)):
        # split test row into input and output columns
        testX, testy = testdata[i, :-1], testdata[i, -1]
        # fit model on history and make a prediction
        yhat = random_forest_forecast(history, testX)
        # store forecast in list of predictions
        predictions.append(yhat)
        # add actual observation to history for the next loop
        history.append(testdata[i])
    # estimate prediction error
    error = mean_absolute_error(testdata[:, -1], predictions)
    return error, testdata[:, -1], predictions

In [ ]:
best_mae = float('inf')  # 初始化为一个无穷大的值，用于保存最低的 MAE
best_params = {}  # 用于保存对应最低 MAE 的参数组合
# 在这里添加对 n_estimators 参数的循环测试
for n_estimators in range(1, 300, 10):
    for max_depth in range(2,12,1):
        for min_samples_leaf in range(3,20,1):
            for n_in in range(1,10):
                traindata = series_to_supervised(trainvalues, n_in=n_in)
                testdata = series_to_supervised(testvalues, n_in=n_in)
                mae, y, yhat = walk_forward_validation(traindata,testdata, 31, n_estimators=n_estimators,max_depth = max_depth,min_samples_leaf = min_samples_leaf)
                print(f'n_estimators={n_estimators}, max_depth={max_depth}, min_samples_leaf={min_samples_leaf}, n_in={n_in}, MAE: {mae}')
                if mae < best_mae:  # 如果当前的 MAE 值比之前记录的最低值小
                    best_mae = mae  # 更新最低 MAE 值
                    best_params['n_estimators'] = n_estimators  # 更新最低 MAE 对应的参数值
                    best_params['max_depth'] = max_depth
                    best_params['min_samples_leaf'] = min_samples_leaf
                    best_params['n_in'] = n_in

n_estimators=1, max_depth=2, min_samples_leaf=3, n_in=1, MAE: 130.41120487202684
n_estimators=1, max_depth=2, min_samples_leaf=3, n_in=2, MAE: 122.93159054663067
n_estimators=1, max_depth=2, min_samples_leaf=3, n_in=3, MAE: 91.43968583681477
n_estimators=1, max_depth=2, min_samples_leaf=3, n_in=4, MAE: 86.74440716067744
n_estimators=1, max_depth=2, min_samples_leaf=3, n_in=5, MAE: 95.37543784492571
n_estimators=1, max_depth=2, min_samples_leaf=3, n_in=6, MAE: 115.48186805267319
n_estimators=1, max_depth=2, min_samples_leaf=3, n_in=7, MAE: 82.73610237479286
n_estimators=1, max_depth=2, min_samples_leaf=3, n_in=8, MAE: 73.3234116267714
n_estimators=1, max_depth=2, min_samples_leaf=3, n_in=9, MAE: 64.38831931952461
n_estimators=1, max_depth=2, min_samples_leaf=4, n_in=1, MAE: 122.35183081142776
n_estimators=1, max_depth=2, min_samples_leaf=4, n_in=2, MAE: 101.51044882267337
n_estimators=1, max_depth=2, min_samples_leaf=4, n_in=3, MAE: 96.43823506361252
n_estimators=1, max_depth=2, min_sam

n_estimators=1, max_depth=2, min_samples_leaf=14, n_in=4, MAE: 94.74604816366937
n_estimators=1, max_depth=2, min_samples_leaf=14, n_in=5, MAE: 112.49707660343606
n_estimators=1, max_depth=2, min_samples_leaf=14, n_in=6, MAE: 105.53393746276747
n_estimators=1, max_depth=2, min_samples_leaf=14, n_in=7, MAE: 94.14195994788152
n_estimators=1, max_depth=2, min_samples_leaf=14, n_in=8, MAE: 70.26484833382887
n_estimators=1, max_depth=2, min_samples_leaf=14, n_in=9, MAE: 78.57691148262535
n_estimators=1, max_depth=2, min_samples_leaf=15, n_in=1, MAE: 127.66228993144276
n_estimators=1, max_depth=2, min_samples_leaf=15, n_in=2, MAE: 114.00975515579435
n_estimators=1, max_depth=2, min_samples_leaf=15, n_in=3, MAE: 116.4637409026288
n_estimators=1, max_depth=2, min_samples_leaf=15, n_in=4, MAE: 113.68971997022078
n_estimators=1, max_depth=2, min_samples_leaf=15, n_in=5, MAE: 104.45603699783297
n_estimators=1, max_depth=2, min_samples_leaf=15, n_in=6, MAE: 105.36959550183151
n_estimators=1, max_d

n_estimators=1, max_depth=3, min_samples_leaf=8, n_in=7, MAE: 57.92482298399747
n_estimators=1, max_depth=3, min_samples_leaf=8, n_in=8, MAE: 50.42987200487522
n_estimators=1, max_depth=3, min_samples_leaf=8, n_in=9, MAE: 40.79478168336245
n_estimators=1, max_depth=3, min_samples_leaf=9, n_in=1, MAE: 117.26460712612953
n_estimators=1, max_depth=3, min_samples_leaf=9, n_in=2, MAE: 90.3779600769047
n_estimators=1, max_depth=3, min_samples_leaf=9, n_in=3, MAE: 67.13445339491487
n_estimators=1, max_depth=3, min_samples_leaf=9, n_in=4, MAE: 73.41001965286898
n_estimators=1, max_depth=3, min_samples_leaf=9, n_in=5, MAE: 53.25238908500168
n_estimators=1, max_depth=3, min_samples_leaf=9, n_in=6, MAE: 68.58726416172645
n_estimators=1, max_depth=3, min_samples_leaf=9, n_in=7, MAE: 52.88526003653343
n_estimators=1, max_depth=3, min_samples_leaf=9, n_in=8, MAE: 40.87309119374943
n_estimators=1, max_depth=3, min_samples_leaf=9, n_in=9, MAE: 54.326002626352235
n_estimators=1, max_depth=3, min_sample

n_estimators=1, max_depth=4, min_samples_leaf=3, n_in=5, MAE: 49.8703122011958
n_estimators=1, max_depth=4, min_samples_leaf=3, n_in=6, MAE: 43.51040444556647
n_estimators=1, max_depth=4, min_samples_leaf=3, n_in=7, MAE: 34.48737684022683
n_estimators=1, max_depth=4, min_samples_leaf=3, n_in=8, MAE: 42.87144039940028
n_estimators=1, max_depth=4, min_samples_leaf=3, n_in=9, MAE: 33.98896586261543
n_estimators=1, max_depth=4, min_samples_leaf=4, n_in=1, MAE: 116.38298659239977
n_estimators=1, max_depth=4, min_samples_leaf=4, n_in=2, MAE: 69.44527162060442
n_estimators=1, max_depth=4, min_samples_leaf=4, n_in=3, MAE: 59.529840084408704
n_estimators=1, max_depth=4, min_samples_leaf=4, n_in=4, MAE: 49.12947566320356
n_estimators=1, max_depth=4, min_samples_leaf=4, n_in=5, MAE: 50.944361110463724
n_estimators=1, max_depth=4, min_samples_leaf=4, n_in=6, MAE: 41.51939012121643
n_estimators=1, max_depth=4, min_samples_leaf=4, n_in=7, MAE: 48.787589440359135
n_estimators=1, max_depth=4, min_samp

n_estimators=1, max_depth=4, min_samples_leaf=14, n_in=8, MAE: 42.20655456014254
n_estimators=1, max_depth=4, min_samples_leaf=14, n_in=9, MAE: 29.677450014602428
n_estimators=1, max_depth=4, min_samples_leaf=15, n_in=1, MAE: 114.59756441285536
n_estimators=1, max_depth=4, min_samples_leaf=15, n_in=2, MAE: 76.77971060171996
n_estimators=1, max_depth=4, min_samples_leaf=15, n_in=3, MAE: 52.00388386716405
n_estimators=1, max_depth=4, min_samples_leaf=15, n_in=4, MAE: 58.921261228298356
n_estimators=1, max_depth=4, min_samples_leaf=15, n_in=5, MAE: 42.78096709811777
n_estimators=1, max_depth=4, min_samples_leaf=15, n_in=6, MAE: 27.124805481006852
n_estimators=1, max_depth=4, min_samples_leaf=15, n_in=7, MAE: 42.69675916589568
n_estimators=1, max_depth=4, min_samples_leaf=15, n_in=8, MAE: 36.22568883765935
n_estimators=1, max_depth=4, min_samples_leaf=15, n_in=9, MAE: 24.373437691293102
n_estimators=1, max_depth=4, min_samples_leaf=16, n_in=1, MAE: 120.99918457753415
n_estimators=1, max_de

...

n_estimators=1, max_depth=11, min_samples_leaf=10, n_in=5, MAE: 44.98108611666675
n_estimators=1, max_depth=11, min_samples_leaf=10, n_in=6, MAE: 41.39617857395257
n_estimators=1, max_depth=11, min_samples_leaf=10, n_in=7, MAE: 46.06866320117097
n_estimators=1, max_depth=11, min_samples_leaf=10, n_in=8, MAE: 40.77477220554781
n_estimators=1, max_depth=11, min_samples_leaf=10, n_in=9, MAE: 35.01922570266725
n_estimators=1, max_depth=11, min_samples_leaf=11, n_in=1, MAE: 104.27584561545982
n_estimators=1, max_depth=11, min_samples_leaf=11, n_in=2, MAE: 77.56903068337694
n_estimators=1, max_depth=11, min_samples_leaf=11, n_in=3, MAE: 74.85325343871172
n_estimators=1, max_depth=11, min_samples_leaf=11, n_in=4, MAE: 51.05731606174043
n_estimators=1, max_depth=11, min_samples_leaf=11, n_in=5, MAE: 41.83957007789084
n_estimators=1, max_depth=11, min_samples_leaf=11, n_in=6, MAE: 41.183526540116425
n_estimators=1, max_depth=11, min_samples_leaf=11, n_in=7, MAE: 60.670961829567865
n_estimators=

n_estimators=11, max_depth=2, min_samples_leaf=4, n_in=6, MAE: 96.55216083848546
n_estimators=11, max_depth=2, min_samples_leaf=4, n_in=7, MAE: 47.23357993556866
n_estimators=11, max_depth=2, min_samples_leaf=4, n_in=8, MAE: 42.30543318273588
n_estimators=11, max_depth=2, min_samples_leaf=4, n_in=9, MAE: 49.44246334557684
n_estimators=11, max_depth=2, min_samples_leaf=5, n_in=1, MAE: 114.9493993693482
n_estimators=11, max_depth=2, min_samples_leaf=5, n_in=2, MAE: 101.77929244938477
n_estimators=11, max_depth=2, min_samples_leaf=5, n_in=3, MAE: 90.29496881448144
n_estimators=11, max_depth=2, min_samples_leaf=5, n_in=4, MAE: 96.42692380583796
n_estimators=11, max_depth=2, min_samples_leaf=5, n_in=5, MAE: 93.81703524535413
n_estimators=11, max_depth=2, min_samples_leaf=5, n_in=6, MAE: 101.7040533650275
n_estimators=11, max_depth=2, min_samples_leaf=5, n_in=7, MAE: 47.19098584662867
n_estimators=11, max_depth=2, min_samples_leaf=5, n_in=8, MAE: 56.890067424617776
n_estimators=11, max_depth

n_estimators=11, max_depth=2, min_samples_leaf=15, n_in=8, MAE: 45.04286226304721
n_estimators=11, max_depth=2, min_samples_leaf=15, n_in=9, MAE: 46.94889699918164
n_estimators=11, max_depth=2, min_samples_leaf=16, n_in=1, MAE: 116.19660227946915
n_estimators=11, max_depth=2, min_samples_leaf=16, n_in=2, MAE: 90.60019059082092
n_estimators=11, max_depth=2, min_samples_leaf=16, n_in=3, MAE: 89.82975860580136
n_estimators=11, max_depth=2, min_samples_leaf=16, n_in=4, MAE: 90.93222109751406
n_estimators=11, max_depth=2, min_samples_leaf=16, n_in=5, MAE: 89.88849209395653
n_estimators=11, max_depth=2, min_samples_leaf=16, n_in=6, MAE: 91.3152240467952
n_estimators=11, max_depth=2, min_samples_leaf=16, n_in=7, MAE: 54.284286295809046
n_estimators=11, max_depth=2, min_samples_leaf=16, n_in=8, MAE: 40.29970674550227
n_estimators=11, max_depth=2, min_samples_leaf=16, n_in=9, MAE: 38.483955169606396
n_estimators=11, max_depth=2, min_samples_leaf=17, n_in=1, MAE: 113.6151591159582
n_estimators=1

n_estimators=11, max_depth=3, min_samples_leaf=10, n_in=2, MAE: 73.49290670807177
n_estimators=11, max_depth=3, min_samples_leaf=10, n_in=3, MAE: 58.27720510708176
n_estimators=11, max_depth=3, min_samples_leaf=10, n_in=4, MAE: 63.279437896748476
n_estimators=11, max_depth=3, min_samples_leaf=10, n_in=5, MAE: 43.38687918674813
n_estimators=11, max_depth=3, min_samples_leaf=10, n_in=6, MAE: 49.7151602234187
n_estimators=11, max_depth=3, min_samples_leaf=10, n_in=7, MAE: 35.57739192275359
n_estimators=11, max_depth=3, min_samples_leaf=10, n_in=8, MAE: 20.71306708182848
n_estimators=11, max_depth=3, min_samples_leaf=10, n_in=9, MAE: 27.607418060091142
n_estimators=11, max_depth=3, min_samples_leaf=11, n_in=1, MAE: 112.05106699503322
n_estimators=11, max_depth=3, min_samples_leaf=11, n_in=2, MAE: 78.46084234388448
n_estimators=11, max_depth=3, min_samples_leaf=11, n_in=3, MAE: 63.601430835281725
n_estimators=11, max_depth=3, min_samples_leaf=11, n_in=4, MAE: 56.38745885884717
n_estimators=

n_estimators=11, max_depth=4, min_samples_leaf=4, n_in=3, MAE: 56.5412202515319
n_estimators=11, max_depth=4, min_samples_leaf=4, n_in=4, MAE: 50.850895662157285
n_estimators=11, max_depth=4, min_samples_leaf=4, n_in=5, MAE: 29.02119882693097
n_estimators=11, max_depth=4, min_samples_leaf=4, n_in=6, MAE: 30.151246824621825
n_estimators=11, max_depth=4, min_samples_leaf=4, n_in=7, MAE: 23.1543055185607
n_estimators=11, max_depth=4, min_samples_leaf=4, n_in=8, MAE: 23.704623613860235
n_estimators=11, max_depth=4, min_samples_leaf=4, n_in=9, MAE: 18.44858442996679
n_estimators=11, max_depth=4, min_samples_leaf=5, n_in=1, MAE: 111.84496549536703
n_estimators=11, max_depth=4, min_samples_leaf=5, n_in=2, MAE: 67.36532518277122
n_estimators=11, max_depth=4, min_samples_leaf=5, n_in=3, MAE: 59.03352577898714
n_estimators=11, max_depth=4, min_samples_leaf=5, n_in=4, MAE: 49.098429880770354
n_estimators=11, max_depth=4, min_samples_leaf=5, n_in=5, MAE: 30.7830723519476
n_estimators=11, max_depth